# Connect to ClickHouse Database

In [174]:
!mkdir -p /home/jupyter/datasphere/project/.clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.clickhouse/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.clickhouse/root.crt

--2024-09-08 17:35:24--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’

/home/jupyter/datas 100%[===================>]   3.50K  --.-KB/s    in 0s      

2024-09-08 17:35:24 (1.05 GB/s) - ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’ saved [3579/3579]



In [175]:
# указываем параметры подключения
CA = '/home/jupyter/datasphere/project/.clickhouse/root.crt' # если вы не меняли путь при скачивании сертификата

VERIFY = True
SECURE = True
USER = 'admin'
PASSWORD = 'hakaton1'
HOST = "rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net" # адрес Clickhouse
PORT = 8443

In [176]:
!pip install clickhouse_connect > /dev/null

In [177]:
# импортируем библиотеку
import clickhouse_connect

# выполняем запрос - в случае успешного выполнения будет показана текущая версия Clickhouse
with clickhouse_connect.get_client(
            host=HOST, port=PORT, username=USER,
            password=PASSWORD, secure=SECURE, verify=VERIFY, ca_cert=CA) as ch_client:
        print(ch_client.command('SELECT version()'))

24.3.10.33


# Генерация IAM токена

In [178]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT API
%pip uninstall jwt
%pip install PyJWT -U

In [179]:
import requests
import json
import time
import jwt
import os

In [180]:
# Замените <идентификатор_сервисного_аккаунта> на ваше значение
service_account_id = "aje286em9u4ibs8572di"

In [181]:
with open('/content/authorized_key.json') as f:
    authorized_key = json.load(f)

In [182]:
# Замените <идентификатор_ключа> на ваше значение
key_id = "ajeo9id7551hpq15fv3o"
private_key = authorized_key['private_key']

In [183]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
iam_token = x['iamToken']

# RAG Pipeline

In [184]:
!pip install -q langchain_community langchain yandexcloud > /dev/null

In [185]:
# llm
folder = 'b1g01stbeeo96j8kfaiv'
model_uri = f'gpt://{folder}/yandexgpt-lite/latest'
model_uri

'gpt://b1g01stbeeo96j8kfaiv/yandexgpt-lite/latest'

In [186]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=iam_token, folder_id=folder)

In [187]:
from langchain_community.vectorstores import Clickhouse, ClickhouseSettings

ch_config = ClickhouseSettings(host=HOST, port=PORT, username=USER, password=PASSWORD)

docsearch = Clickhouse(embedding=embeddings, config=ch_config, verify=VERIFY, ca_cert=CA)

In [188]:
retriever = docsearch.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)

In [189]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """
Ты - генератор интерфейсов по запросу пользователя на естественном языке. Получая описание интрефейса, ты генерируешь код.
Твой ответ должен содержать только код и не должен содержать описания или другого посторонего текста.
Ты обязан использовать дизанйн компоненты системы НЛМК.
Ответьте на вопрос, основываясь только на следующем контексте:
{context}

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nТы - генератор интерфейсов по запросу пользователя на естественном языке. Получая описание интрефейса, ты генерируешь код.  \nТвой ответ должен содержать только код и не должен содержать описания или другого посторонего текста.\nТы обязан использовать дизанйн компоненты системы НЛМК. \nОтветьте на вопрос, основываясь только на следующем контексте:\n{context}\n\nВопрос: {question}\n'))])

In [190]:
from langchain_community.llms import YandexGPT

llm = YandexGPT(iam_token=iam_token, model_uri=model_uri)

In [191]:
# Chain
chain = prompt | llm

In [192]:
question = "Создай карточку продукта с изображением, названием, ценой и кнопкой купить."

# Использовать релевантные для запроса документы

In [193]:
results = docsearch.similarity_search(query=question, k=1)
print(len(results))
for doc in results: print(f"* {doc.page_content} [{doc.metadata}]")

1
* Card
Компонент Card является функциональным компонентом, который предназначен для отображения карточки с различными элементами интерфейса, такими как изображение, заголовок, описание, значки, селектор и группа кнопок.
Card по умолчанию
import { Button, Card, Typography, Select } from '@nlmk/ds-2.0';

export default  App = () => (
  <>
    <div style={{
      backgroundColor: 'var(--steel-20)',
      width: 'fit-content',
      padding: '20px'
  }}>
    <Card>
      <div
        style={{
          height: '200px',
          width: '300px',
          display: 'flex',
          flexDirection: 'column',
          gap: '10px'
        }}
      >
        <Typography variant='Heading3'>
          Заголовок
        </Typography>

        <Select
          label="Выберите язык программирования"
          options={[
            {
              label: 'C++',
              value: 'C++'
            },
            {
              label: 'C#',
              value: 'C#'
            }]}
        />



In [194]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke(question)
print(response)

```
import { Button, Card, Typography } from '@nlmk/ds-2.0';

export default function App() {
  return (
    <div style={{ backgroundColor: 'var(--steel-20)', width: 'fit-content', padding: '20px' }}>
      <Card>
        <img src="product-image.jpg" alt="Product image" width="100%" height="150px" />
        <div style={{ height: '200px', width: '300px', display: 'flex', flexDirection: 'column', gap: '10px' }} >
          <Typography variant='Heading3'>
            Название продукта
          </Typography>
          <Select label="Выберите язык программирования" options={[{ label: 'C++', value: 'C++' }, { label: 'C#', value: 'C#' }]} />
          <div style={{ display: 'flex', gap: '8px', marginTop: 'auto' }} >
            <Button>Купить</Button>
            <Button variant="secondary">Подробнее</Button>
          </div>
        </div>
      </Card>
    </div>
  );
}
```
